<https://huggingface.co/tokyotech-llm/Swallow-13b-instruct-hf>

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\hamada\workspace\test\llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "tokyotech-llm/Swallow-7b-instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")

model-00003-of-00003.safetensors: 100%|██████████| 3.77G/3.77G [03:31<00:00, 17.8MB/s]
generation_config.json: 100%|██████████| 203/203 [00:00<00:00, 204kB/s]


In [4]:
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}


def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

In [9]:
# Example usage
instruction_example = "入力内用はとあるソフトウェアのインターフェースを説明した文章です。入力内用を英語に翻訳してください。また、応答には翻訳した文章のみを出力してください"
input_example = "View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。"
prompt = create_prompt(instruction_example, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=128,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
入力内用はとあるソフトウェアのインターフェースを説明した文章です。入力内用を英語に翻訳してください。また、応答には翻訳した文章のみを出力してください

### 入力:
View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。

### 応答:The method parameter can accept a ViewModel as a second argument, if a View's constructor is called with that second argument.
